<a href="https://colab.research.google.com/github/Aleksandr62aa/YOLOv8_Tracking/blob/main/Yolo_BotSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Detect YOLO + BoT-SORT an video**

In [ ]:
# Setup YOLO
!pip install ultralytics
!pip install lapx
from ultralytics import YOLO
import ultralytics
ultralytics.__version__

In [ ]:
# results[0].boxes

In [2]:
# Importing libraries
import torch
import numpy as np
import cv2
import time

In [4]:
# Detect YOLO + Bot-Sort
class YoloBotSort:

    def __init__(self):

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

    def load_model(self):

        model = YOLO("yolov8n.pt")  # load a pretrained YOLOv8n model
        model.fuse()

        return model

    def predict(self, frame):

        results = self.model.track(frame, classes=0, verbose=False, persist=True)

        return results

    def get_results(self, results):

        detect_track_list = []
        # Extract detections
        for result in results[0]:
            bbox = result.boxes.xyxy.cpu().numpy()
            confidence = result.boxes.conf.cpu().numpy()
            ids = result.boxes.id.cpu().numpy()

            merged_detect_track = [bbox[0][0], bbox[0][1], bbox[0][2], bbox[0][3], confidence[0], ids[0]]
            detect_track_list.append(merged_detect_track)

        return np.array(detect_track_list)

    def draw_bounding_boxes_with_id(self, img, bboxes, ids):

        for bbox, id_ in zip(bboxes, ids):
            cv2.rectangle(img,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),(0,0,255),2)
            cv2.putText(img, "ID: " + str(id_), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        return img

    def __call__(self):

        video_path = '/content/test3.mp4'

        cap = cv2.VideoCapture(video_path)
        assert cap.isOpened()

        # Get the video properties
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        # Output the video path
        video_out_path = 'output_BotSort.avi'
        out = cv2.VideoWriter(video_out_path, fourcc, 10, (frame_width, frame_height))

        counter = 0
        start_time = time.perf_counter()

        while cap.isOpened():

            ret, frame = cap.read()
            if ret:
                results = self.predict(frame)
                detect_track_list = self.get_results(results)

                boxes_track = detect_track_list[:,:-2]
                boxes_conf = detect_track_list[:,-2].astype(int)
                boxes_ids = detect_track_list[:,-1].astype(int)

                frame = self.draw_bounding_boxes_with_id(frame, boxes_track, boxes_ids)

                # Update FPS and place on frame
                current_time = time.perf_counter()
                elapsed = (current_time - start_time)
                counter += 1
                if elapsed > 1:
                    fps = counter / elapsed
                    counter = 0
                    start_time = current_time

                cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

                out.write(frame)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break

        cap.release()
        out.release()
        cv2.destroyAllWindows()


In [ ]:
detector = YoloBotSort()
detector()